In [1]:
import os

In [3]:
#%pwd
#os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL: str
    local_data_file:Path
    unzip_dir:Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH
            ):

            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])


               
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [14]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import ssl
import requests
import urllib.error as URlError


# Disable SSL verification
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

#with request.urlopen("https://google.com", context=ssl_context) as response:
 #   print(response.read())
class DataIngestion:
    
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    '''
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
    '''

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                response = requests.get(self.config.source_URL, 
                                        stream=True, 
                                        verify=False)  # Bypass SSL certificate verification
                
                with open(self.config.local_data_file, 'wb') as file:
                    for chunk in response.iter_content(chunk_size=8192): 
                        file.write(chunk)

                logger.info(f"{self.config.local_data_file} downloaded successfully!")
                logger.info(f"Response headers: \n{response.headers}")
            except URLError as e:
                logger.error(f"Error downloading file: {e}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [15]:
try:
    config = ConfigurationManager()
    #config.get_data_ingestion_config().source_URL
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-24 14:11:51,854: INFO: common: content : {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/shilpasngh/Datasets/raw/refs/heads/main/chicken_disease.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}]
[2025-01-24 14:11:51,855: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-01-24 14:11:51,857: INFO: common: content : {'key': 'value'}]
[2025-01-24 14:11:51,857: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-24 14:11:51,859: INFO: common: created directory at: artifacts]
[2025-01-24 14:11:51,859: INFO: common: created directory at: artifacts/data_ingestion]


/Users/shilpasingh/Desktop/PROJECTS/CHICKEN_DISEASE_CLASSIFICATION/Chicken-Disease-Classification-Project/env3.12/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'github.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/shilpasingh/Desktop/PROJECTS/CHICKEN_DISEASE_CLASSIFICATION/Chicken-Disease-Classification-Project/env3.12/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[2025-01-24 14:11:52,300: INFO: 617547180: artifacts/data_ingestion/data.zip downloaded successfully!]
[2025-01-24 14:11:52,301: INFO: 617547180: Response headers: 
{'Connection': 'keep-alive', 'Content-Length': '9546433', 'Cache-Control': 'max-age=300', 'Content-Security-Policy': "default-src 'none'; style-src 'unsafe-inline'; sandbox", 'Content-Type': 'application/zip', 'ETag': 'W/"10a901294a979df4cb3a4cda0cc3e058e3da7e22fa3bf6bffec75c02d62442f2"', 'Strict-Transport-Security': 'max-age=31536000', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'deny', 'X-XSS-Protection': '1; mode=block', 'X-GitHub-Request-Id': '55B8:2BF01E:54828:5DDF3:6793E0B9', 'Accept-Ranges': 'bytes', 'Date': 'Fri, 24 Jan 2025 19:11:52 GMT', 'Via': '1.1 varnish', 'X-Served-By': 'cache-yyz4576-YYZ', 'X-Cache': 'HIT', 'X-Cache-Hits': '0', 'X-Timer': 'S1737745912.220811,VS0,VE1', 'Vary': 'Authorization,Accept-Encoding,Origin', 'Access-Control-Allow-Origin': '*', 'Cross-Origin-Resource-Policy': 'cross-origin',